In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import os
import glob
warnings.filterwarnings('ignore')

class DetectionStatisticAnalyzer:
    
    file_list = []
    optimal_parameter = []  
    
    def __init__(self, data_path, save_results):
        self.N_phi = 8
        self.N_w = 512
        self.ref_value = -100                # We assign some ref det statistic value to compare.
        self.save_results = save_results
        
        """ These lists will be used to store the values calculated by algorithm """
        
        self.phase_list = []
        self.freq_list = []
        self.det_stat_list = []

        self.duty_cycle = 0.1               # Duty cycle value may vary for different observations
        self.acceleration = 0
        
        df = pd.read_csv(data_path, delimiter='    ')
        self.time = np.array(df['time_sec'])
        self.datastream = np.array(df['intensity'])
        self.sample_time = df['time_sec'][1] - df['time_sec'][0]     # Time resolution of the data
        
        ''' Lower and upper frequency bound are calculated based on data '''
        self.low_limit_freq = 2/(max(df['time_sec']))
        self.up_limit_freq = 1/(2*self.sample_time)
        print('\nYour data is being searched in the total frequency range of',self.low_limit_freq, 'Hz to ', self.up_limit_freq, 'Hz\n')

        """
        Removing pre-existing files in the target folder so that new files won't get confused with old files. 
        """
        files_removing = glob.glob(save_results + '/*')
        for files in files_removing:
            os.remove(files)

        
    def pulse_phase(self, p0, t, w0):    
        """
        Calculates and returns pulse phase at any given time 't' when acceleration 'a', initial 
        phase 'p0' and angular frequency 'w0' are known.
        """
        phase = np.array(p0 + 2*np.pi*w0*t + 0.5*self.acceleration*t** 2)
        return phase

    def von_mises_profile(self, phase_angle):
        """
        Calculates von Mises profile based on concentration function K. For the value of pulse_phase it makes a call from 
        pulse_phase function defined above. If duty cycle has fixed value we can keep value of K fixed instead of having to 
        calculate evertime. This will reduce one more calculation.
        """
        k = (np.log(2)) / (2 * (np.sin((np.pi * self.duty_cycle) / 2)) ** 2)
        y = np.exp(-2*k*(np.sin(phase_angle/2))**2)
        return y
    
    def detection_statistic(self, phaseangle):
        """
        Calculates the detection statistic based on eqn 5 in Smith K, 2016 paper. I_k is von_mises profile calculated for 
        various phase angles. Phase angle itself is calculated from pulse_phase function for the time series.
        """
        I_k = self.von_mises_profile(phaseangle)                                        # Intensity 
        eta_neg_sqr = 1 / (self.sample_time * (np.sum(I_k ** 2)))                       # Gives a noise parameter
        det_stat = eta_neg_sqr * self.sample_time * (np.sum(self.datastream * I_k))     # Eqn 5 in K Smith, 2016 paper
        return det_stat
        
    def ComputeDetStat(self, low_freqlimit, high_freqlimit):
        
        ''' Will do the calculation for detection statics in a matrix of shape given by self.N_phi and self.N_w values. 
        Calculation will be done in the range of frequencies given by low_freqlimit and high_freqlimit. We don't specify range 
        for phase because we search over all the the pahse space of 0 to 2*pi. Detection statistics will be stored in a csv file
        for future reference in the folder specified as target path. Will also create a 'pcolormesh' plot for visualization '''
        
        self.rest_phase = np.linspace(0, np.pi*2, self.N_phi)                 # Creating a grid of phase and frequency search space
        self.freq = np.linspace(low_freqlimit, high_freqlimit, self.N_w)
                    
        for ind1 in range(len(self.rest_phase)):
            p0_val = round(self.rest_phase[ind1], 12)
            for ind2 in range(len(self.freq)):
                freq_value = round(self.freq[ind2], 12)
                self.phase_list.append(p0_val)
                self.freq_list.append(freq_value)
                
                phase_val = self.pulse_phase(p0_val, self.time, freq_value)  # Calculate phase and detection statistics
                det_stat_val = self.detection_statistic(phase_val)
                
                self.det_stat_list.append(round(det_stat_val, 12))
                
        data = {'Phase': self.phase_list, 'Frequency': self.freq_list, 'Detection Statistic': self.det_stat_list}
        phasefreqdf = pd.DataFrame(data, index=np.arange(len(self.det_stat_list)))                    
        phasefreqdf.to_csv(self.save_results + '/L_freq = {}__U_freq = {}__N_phase = {}_and_N_freq = {}'.format(low_freqlimit, high_freqlimit, self.N_phi, self.N_w), sep= '\t', mode = 'w', columns=['Phase','Frequency','Detection Statistic'], header = ['Phase','Frequency','Detection Statistic'], index=False)
        self.plot_detection_statistic(self.N_phi, self.N_w, low_freqlimit, high_freqlimit, phasefreqdf)

    def run_detection(self):
        ''' This section is like an actual manufacturing plant. First while loop part will perform a low key search and combine 
        all the results from this low key search at serveral different frequency range. Next, we make a call for several methods
        as necessary  to get diagnostic plots and do more intense search after finding intial approximate maximum detection.  '''
        
        self.low_freq = self.low_limit_freq        # Creating frequency search boundary
        self.high_freq = 10*self.low_freq
    
        while self.high_freq <= (self.up_limit_freq)/2:
                freq = np.linspace(self.low_freq, self.high_freq, self.N_w)
                print('Frequency search range', self.low_freq, '--', self.high_freq)
                self.ComputeDetStat(self.low_freq, self.high_freq)
                self.phase_list.clear()
                self.freq_list.clear()
                self.det_stat_list.clear() 
                DetectionStatisticAnalyzer.file_list.append('L_freq = {}__U_freq = {}__N_phase = {}_and_N_freq = {}'.format(self.low_freq, self.high_freq, self.N_phi, self.N_w))
                self.low_freq = round(self.high_freq, 12) + 1e-2
                self.high_freq = 10*self.low_freq
            
        self.CombinedDataFrame()
        self.MaxvaluePlot()      
        
    def plot_detection_statistic(self, N_phase, N_freq, l_freq, h_freq, phasefreq_df):

        ''' Creates a color mesh plot for matrix of phase and frequency values calculated in 'ComputeDetStat' method above.
        Detection statistic will be color coded. '''
        
        fig, phasefreqmap = plt.subplots(figsize=(15, 10), layout='constrained')
        X = np.linspace(l_freq, h_freq, N_freq + 1)
        Y = np.linspace(0, 2*np.pi, N_phase + 1)
        Z = np.array(phasefreq_df['Detection Statistic']).reshape(N_phase, N_freq)
        pcm = phasefreqmap.pcolormesh(X, Y, Z, vmin=Z.min(), vmax=Z.max())
        phasefreqmap.set_xlabel('Frequency(Hz)', fontweight='bold', fontsize=20)
        phasefreqmap.set_ylabel('Phase', fontweight='bold', fontsize=20)
        phasefreqmap.yaxis.set_tick_params(labelsize=20)
        phasefreqmap.xaxis.set_tick_params(labelsize=20)
        phasefreqmap.set_title('Detection statistic for N_phase = {} and N_freq = {} combination'.format(N_phase, N_freq), fontweight='bold', fontsize=20)
        fig.colorbar(pcm, ax=phasefreqmap)
        plt.savefig(self.save_results + '/L_freq = {}__U_freq = {}__N_phase = {}_and_N_freq = {}.pdf'.format(l_freq, h_freq, N_phase, N_freq))  # Only if you want to save your figure.
        plt.close()
    
    def CombinedDataFrame(self):

        ''' This method will combine detection statistics data calculated for different frequency ranges into one concatenated
        file which will be used to create overall detection statistic vs frequency plot.'''
        
        self.fileList = DetectionStatisticAnalyzer.file_list
        for k in range(len(self.fileList)):
            file_path = self.save_results + self.fileList[k]
            df = pd.read_csv(file_path, delimiter = '\t')
            
            if k == 0:
                df_whole = df
            else:
                df_whole = pd.concat([df_whole, df], ignore_index=True)
                
        df_max = df_whole[df_whole['Detection Statistic'] >= self.ref_value]       
        df_max.to_csv(self.save_results +'detection_val'+ str(self.ref_value), sep= '\t', mode = 'w', columns=['Phase','Frequency','Detection Statistic'], header = ['Phase','Frequency','Detection Statistic'], index=False)
        self.df_max = df_max
        self.MaxvaluePlot()
        max_det_value = max(df_max['Detection Statistic'])
        lowest_det_value = min(df_whole['Detection Statistic'])
        optimal_df = df_max.query('`Detection Statistic`==@max_det_value')
        optimal_freq = max(optimal_df['Frequency'])
        
        print('\nLowest DetStatValue is: ', lowest_det_value)
        print('Ref DetStatValue = ', self.ref_value)
        print('Warning! If Lowest DetStatValue <= Ref DetStatValue change Lowest DetStatValue to value lower than Ref DetStatValue and re-run the analysis.')
        print('\nApproximate optimal frequency is: ', optimal_freq)
        
    def MaxvaluePlot(self):

        ''' Creates a plot for combined file with detection statistics calculated for all the frequency range. This plot can
        be visually inspected to see what is the frequency range to look for to do an intensive search. So, this plot will 
        basically guide us narrow down our search to particular frequency range. '''
        
        fig, ax = plt.subplots(figsize=(12, 10), layout='constrained')
        self.detstat = self.df_max['Detection Statistic']
        self.freq = self.df_max['Frequency']
        ax.semilogx(self.freq, self.detstat)
        ax.set_xlabel('Frequency Range', fontweight = 'bold', size = 28)
        ax.set_ylabel('Detection Statistic', fontweight = 'bold', size = 28)
        ax.set_title('Detection statistics plot for sim_pulse_2 data' , fontweight = 'bold', size = 20)
        ax.xaxis.set_tick_params(labelsize=24)
        ax.yaxis.set_tick_params(labelsize=24)
        plt.savefig(self.save_results + 'det_stat_plot.pdf')
        plt.close()

det_stat = DetectionStatisticAnalyzer(data_path='/home/ak/research/emmanuel/proj_algorithm/sim_pulse_02_updated.txt', save_results = '/home/ak/Desktop/testresults/')
det_stat.run_detection()


Your data is being searched in the total frequency range of 2.0000012180007416 Hz to  73227.88517867605 Hz

Frequency search range 2.0000012180007416 -- 20.000012180007417
Frequency search range 20.010012180007003 -- 200.10012180007004
Frequency search range 200.11012180007 -- 2001.1012180007
Frequency search range 2001.1112180007 -- 20011.112180007

Lowest DetStatValue is:  0.001385351726
Ref DetStatValue =  -100
Warning! If Lowest DetStatValue <= Ref DetStatValue change Lowest DetStatValue to value lower than Ref DetStatValue and re-run the analysis.

Approximate optimal frequency is:  114.460402352565
